# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [16]:
# Write your code below.
from dotenv import load_dotenv
import os

load_dotenv()




True

In [17]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [18]:
import os
from glob import glob

# Write your code below.
#get the path to the prices data
prices_data_path = os.getenv('PRICE_DATA')
print("PRICE_DATA path:", prices_data_path)
#list all the parquet files
all_parquet_files =glob(os.path.join(prices_data_path, "**/*.parquet"), recursive=True)
print("Found files:", all_parquet_files)

PRICE_DATA path: ../../05_src/data/prices/
Found files: ['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [19]:
# Write your code below.

# Read the parquet files
ddf = dd.read_parquet(all_parquet_files)

# Sort values to ensure correct order
ddf = ddf.sort_values(by=["ticker", "date"])
print(ddf.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [21]:
# Write your code below.
# add lag features
ddf['Close_lag_1'] = ddf.groupby('ticker')['Close'].shift(1)
ddf['Adj_Close_lag_1'] = ddf.groupby('ticker')['Adj Close'].shift(1)

#Add returns
ddf['returns'] = (ddf['Close'] / ddf['Close_lag_1']) -1

# Add high and low range
ddf['high_range'] = ddf['High'] - ddf['Low']

#Assign final result to dd_feat
dd_feat = ddf
print(ddf.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns',
       'high_range'],
      dtype='object')


C:\Users\afsan\AppData\Local\Temp\ipykernel_6868\1029951446.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Close_lag_1'] = ddf.groupby('ticker')['Close'].shift(1)
C:\Users\afsan\AppData\Local\Temp\ipykernel_6868\1029951446.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Adj_Close_lag_1'] = ddf.groupby('ticker')['Adj Close'].shift(1)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.